In [1]:
import os
os.environ["HIP_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:

# -*- coding: utf-8 -*-

import pandas as pd
from datasets import Dataset, DatasetDict
import evaluate
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import classification_report, confusion_matrix
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import numpy as np
import torch
from torch.nn.functional import softmax
from pathlib import Path
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1) LOCAL CSV PATH
# ----------------------------
csv_path = Path(r"AI_Human.csv")
df = pd.read_csv(csv_path)
df['generated'] = df['generated'].astype(int)

df["raw_index"] = df.index

# Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)

# ----------------------------
# 2) Train / Val / Test split
# ----------------------------
dataset_split = dataset.train_test_split(test_size=0.1, seed=42)
train_val_split = dataset_split['train'].train_test_split(test_size=0.1, seed=42)

dataset = DatasetDict({
    "train": train_val_split["train"],
    "validation": train_val_split["test"],
    "test": dataset_split["test"]
})

# ----------------------------
# Verification:
# ----------------------------
if torch.cuda.is_available():
    print("CUDA is available")
    print("Device count:", torch.cuda.device_count())
    print("Current device index:", torch.cuda.current_device())
    print("Current device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available")

if torch.version.hip:
    print("Using ROCm backend")


# ----------------------------
# 3) Tokenization
# ----------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)
model.to(device)

def tokenize_fn(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize_fn, batched=True)
cols_to_remove = [c for c in ["text", "__index_level_0__"] if c in dataset["train"].column_names and c != "raw_index"]
dataset = dataset.remove_columns(cols_to_remove)
if "generated" in dataset["train"].column_names:
    dataset = dataset.rename_column("generated", "labels")
dataset.set_format("torch")
dataset.save_to_disk("final_split_dataset")
print("Tokenized and split dataset saved to 'final_split_dataset'")

# ----------------------------
# 4) Training setup
# ----------------------------
training_args = TrainingArguments(
    output_dir="finetuned-distilbert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    bf16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="binary")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# ----------------------------
# 5) Train
# ----------------------------
checkpoint_dir = Path(training_args.output_dir)
checkpoints = [d for d in checkpoint_dir.glob("checkpoint-*") if d.is_dir()]

if checkpoints:
    latest_checkpoint = sorted(checkpoints, key=os.path.getmtime)[-1]
    print(f"Resuming from checkpoint: {latest_checkpoint}")
    trainer.train(resume_from_checkpoint=str(latest_checkpoint))
else:
    print("No checkpoints found. Starting training from scratch.")
    trainer.train()

# ----------------------------
# 6) Evaluate
# ----------------------------
eval_results = trainer.evaluate(eval_dataset=dataset["test"])
print("Test Evaluation Metrics:")
for metric, value in eval_results.items():
    if metric != "eval_runtime":
        print(f"{metric}: {value:.4f}")

preds = trainer.predict(dataset["test"])
y_pred = np.argmax(preds.predictions, axis=1)
y_true = preds.label_ids

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Human", "AI"]))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# ----------------------------
# 7) Analyze FPs and FNs
# ----------------------------

original_test_df = pd.read_csv(csv_path)
test_indices = dataset["test"]["raw_index"]
original_test_split = original_test_df.iloc[test_indices].reset_index(drop=True)

# Identify FPs and FNs
fp_indices = np.where((y_pred == 1) & (y_true == 0))[0]  # Human predicted as AI
fn_indices = np.where((y_pred == 0) & (y_true == 1))[0]  # AI predicted as Human

print("\n False Positives (Predicted: AI, Actual: Human):")
for idx in fp_indices:
    print(f"→ {original_test_split.iloc[idx]['text']}")

print("\n False Negatives (Predicted: Human, Actual: AI):")
for idx in fn_indices:
    print(f"→ {original_test_split.iloc[idx]['text']}")

# ----------------------------
# 8) Save model
# ----------------------------
save_path = Path(r"distilbert-human-ai-model")
save_path.mkdir(parents=True, exist_ok=True)

trainer.save_model(str(save_path))
tokenizer.save_pretrained(str(save_path))
print(f"\n Model & tokenizer saved to: {save_path}")

# ----------------------------
# 9) Reload for inference
# ----------------------------
tokenizer = DistilBertTokenizerFast.from_pretrained(str(save_path))
model = DistilBertForSequenceClassification.from_pretrained(str(save_path))
model.eval()

def predict_text_source(text: str, model, tokenizer):
    model.to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1).squeeze()
        pred = torch.argmax(probs).item()
    label = "Human" if pred == 0 else "AI"
    confidence = probs[pred].item()
    return f"{label} (Confidence: {confidence:.2f})"

# ----------------------------
# 10) Re-check predictions on FP and FN examples
# ----------------------------
print("\n Re-checking predictions using `predict_text_source()` function:")

print("\n[False Positives]")
for idx in fp_indices[:3]:  # Top 3 only
    text = original_test_split.iloc[idx]['text']
    print(f"\n{text}\n→ Prediction: {predict_text_source(text, model, tokenizer)}")

print("\n[False Negatives]")
for idx in fn_indices[:3]:
    text = original_test_split.iloc[idx]['text']
    print(f"\n{text}\n→ Prediction: {predict_text_source(text, model, tokenizer)}")


CUDA is available
Device count: 1
Current device index: 0
Current device name: AMD Radeon RX 9060 XT
Using ROCm backend


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Saving the dataset (1/1 shards): 100%|██████████| 48724/48724 [00:00<00:00, 3234158.92 examples/s]
/tmp/ipykernel_3056/2722805524.py:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Tokenized and split dataset saved to 'final_split_dataset'
Resuming from checkpoint: finetuned-distilbert/checkpoint-24668


Epoch,Training Loss,Validation Loss,Accuracy,F1
3,0.000500,0.002267,0.999270,0.999019


Test Evaluation Metrics:
eval_loss: 0.0013
eval_accuracy: 0.9994
eval_f1: 0.9992
eval_samples_per_second: 949.7990
eval_steps_per_second: 29.6890
epoch: 3.0000

Classification Report:
              precision    recall  f1-score   support

       Human       1.00      1.00      1.00     30442
          AI       1.00      1.00      1.00     18282

    accuracy                           1.00     48724
   macro avg       1.00      1.00      1.00     48724
weighted avg       1.00      1.00      1.00     48724

Confusion Matrix:
[[30431    11]
 [   20 18262]]

 False Positives (Predicted: AI, Actual: Human):
→ Dear, Manager

Have you ever had a dream job? I have always wanted to be a professional gamer. Gaming is so unique in so many ways, like the fact that you can create your own game and have such a large fan base growing ever years is incredible. I have been gaming ever since I turn five years old. Gaming is such an important part of my life because my dad gave me my first console to pla

In [5]:
save_path = Path(r"distilbert-human-ai-model")
save_path.mkdir(parents=True, exist_ok=True)

trainer.save_model(str(save_path))
tokenizer.save_pretrained(str(save_path))

('distilbert-human-ai-model/tokenizer_config.json',
 'distilbert-human-ai-model/special_tokens_map.json',
 'distilbert-human-ai-model/vocab.txt',
 'distilbert-human-ai-model/added_tokens.json',
 'distilbert-human-ai-model/tokenizer.json')

In [6]:
# inference.py

import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.nn.functional import softmax

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load saved model & tokenizer
model_path = "distilbert-human-ai-model"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path).to(device)
model.eval()

# Prediction function
def predict_text_source(text: str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1).squeeze()
        pred = torch.argmax(probs).item()
    label = "Human" if pred == 0 else "AI"
    confidence = probs[pred].item()
    return f"{label} (Confidence: {confidence:.2f})"

# Example usage
if __name__ == "__main__":
    example_text = "This is a piece of generated content using AI."
    prediction = predict_text_source(example_text)
    print(f"Prediction: {prediction}")


Prediction: AI (Confidence: 1.00)
